In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import cv2
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches
from pathlib import Path
import sys
sys.path.append("/home/psa_images/SemiF-AnnotationPipeline")
sys.path.append("/home/psa_images/SemiF-AnnotationPipeline/segment")
from semif_utils.utils import cutoutmeta2csv
from semif_utils.utils import apply_mask

/home/mkutuga/anaconda3/envs/semif/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [84]:
def read_metadata(path):
    with open(path, 'r') as f:
        data = json.loads(f.read())
    return data

def get_detection_data(jsonpath):
    meta = read_metadata(jsonpath)
    imgwidth, imgheight = meta['width'], meta['height']
    bboxes = meta["bboxes"]
    boxes = []
    labels = []
    for box in bboxes:
        
        # top_left = box["local_coordinates"]["top_left"]
        # bottom_right = box["local_coordinates"]["bottom_right"]
        
        # print(box["local_coordinates"].keys())
        x1 = box["local_coordinates"]["top_left"][0] # * 9560
        y1 = box["local_coordinates"]["top_left"][1] # * 6368
        
        x2 = box["local_coordinates"]["bottom_right"][0] # * 9560
        y2 = box["local_coordinates"]["bottom_right"][1] # * 6368
        
        # xywh = box["local_coordinates"]["xywh"]
        # w = xywh[2] * imgwidth
        # h = xywh[3] * imgheight
        # x1 = xywh[0] * imgwidth - w/2
        # y1 = xywh[1] * imgheight - h/2
        # x2 = x1 + w
        # y2 = y1 + h
        
        # x1, y1 = int(top_left[0]*imgwidth), int(top_left[1]*imgheight)
        # x2, y2 = int(bottom_right[0]*imgwidth), int(bottom_right[1]*imgheight)
        # print([x1, y1, x2, y2])
        boxes.append([x1, y1, x2, y2])
        if box["cls"] == "plant":
            class_id = box["cls"]
            common_name = box["cls"]
        else:
            class_id = str(box["cls"]["class_id"])
            common_name = box["cls"]["common_name"]

        label =  f"{common_name} ({class_id})"
        labels.append(label)

    return boxes, labels

def batch_df(df, batch_id, cutout_dir, batch_dir):
    df = pd.read_csv(Path(cutout_dir, batch_id, batch_id + ".csv"))
    df["state_id"] = df.batch_id.str.split("_", expand=False).str[0]
    df["date"] = df.batch_id.str.split("_", expand=False).str[1]
    df["cutout_paths"] = cutout_dir + "/" + batch_id + "/" + df["cutout_id"] + ".png"
    df["image_paths"] = batch_dir + "/images/" + df["image_id"] + ".jpg"
    df["meta_paths"] = batch_dir + "/metadata/" + df["image_id"] + ".json"
    df["semantic_masks"] = batch_dir + "/meta_masks/semantic_masks/" + df["image_id"] + ".png"
    df["instance_masks"] = batch_dir + "/meta_masks/instance_masks/" + df["image_id"] + ".png"
    return df

def review_data(df, batch_id):
    print("\nBatch: ", batch_id)
    print("\nTotal number of cutouts by species")
    print(df.groupby(["common_name"])["cutout_id"].nunique())
    print(len(df))



In [81]:
cutout_dir = Path( "/mnt/research-projects/s/screberg/longterm_images/semifield-cutouts")
batch_dir = Path("/mnt/research-projects/s/screberg/longterm_images/semifield-developed-images")
cutout_batches = [x.stem for x in cutout_dir.glob("*")]
project_bathces =  [x.stem for x in batch_dir.glob("*")]
common_batches = list(set(cutout_batches).intersection(project_bathces))
uncommon_batches = list(set(cutout_batches).symmetric_difference(project_bathces))

## Check cutouts json data schema from Summer Weeds 2022 and Cover crops 2023

In [111]:
first = common_batches[2]
last = common_batches[40]
compared_batches = [first, last]
compared_batches

keys = []
cls_keys = []
props = []
dict_keys = []

for batch in compared_batches:
    metadata = [x for x in Path(cutout_dir, batch).glob("*.json")][:50]
    for meta in metadata:
        data = read_metadata(meta)
        keys.append(data.keys())
        cls_keys.append(data["cls"].keys())
        props.append(data["cutout_props"].keys())
        for key, value in data.items():
            if type(value) is dict:
                dict_keys.append(key)

print(keys[0])
print(keys[-1])
print()
print(cls_keys[0])
print(cls_keys[-1])
print()
print(props[0])
print(props[-1])

In [113]:
weeds = read_metadata(metadata[0])
ccs = read_metadata(metadata[-1])


{'blob_home': 'data',
 'data_root': 'semifield-cutouts',
 'season': 'cool_season_covers_2022_2023',
 'batch_id': 'TX_2023-05-10',
 'image_id': 'TX_1683746532',
 'cutout_id': 'TX_1683746532_4',
 'cutout_path': 'TX_2023-05-10/TX_1683746532_4.png',
 'cls': {'class_id': 38,
  'USDA_symbol': 'AVSA',
  'EPPO': 'AVESA',
  'group': 'monocot',
  'class': 'Liliopsida',
  'subclass': 'Commelinidae',
  'order': 'Cyperales',
  'family': 'Poaceae',
  'genus': 'Avena',
  'species': 'sativa',
  'common_name': 'Oats',
  'authority': 'Linnaeus',
  'growth_habit': 'graminoid',
  'duration': 'annual',
  'collection_location': 'NC, TX',
  'category': 'cool season cover crop',
  'collection_timing': 'winter',
  'multi_species_USDA_symbol': None,
  'link': 'https://plants.usda.gov/home/plantProfile?symbol=AVSA',
  'note': None,
  'hex': '#44975e',
  'rgb': [68, 151, 94]},
 'cutout_num': 4,
 'bbox': [2703, 1111, 2754, 1151],
 'is_primary': False,
 'datetime': '2023:05:10 13:19:01',
 'shape': [40, 51, 3],
 'cu